# TODO

## Major
- [x] create term-by-document matrix (calculate words frequncies for each term-document pair)
 - [ ] check that it's actually correct - especially if we don't map terms to wrong documents
- [x] convert term-by-document frequencies to tf-idf (calcualte tf-idf for each term-document pair)
 - [ ] check
- [ ] we may need actual (numpy?) matrix?
- [ ] LSI magic

### Minor
- [x] remove numbers from terms - done but not sure if it's good thing to do, maybe it's also important for relevancy of docs,
like for example when there is year written?

In [24]:
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer

In [25]:
np.random.seed(42)

In [26]:
bp_data = pd.read_csv("articles.csv", header=0)

In [27]:
bp_data.head(1)

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."


In [28]:
def preprocess_docs(use_lemmatizer = True):
    '''Tokenize and preprocess documents
    
    Parameters
    ----------
    use_lemmatizer : bool
                     Uses lemmazizer if True, othrerwise uses stemmer.
    '''
    preproccessed_docs = []
    
    # English stop words list
    en_stop = set(stopwords.words('english'))
    
    # Word tokenizer that removes punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    
    # lemmatizer / Stemmer
    if use_lemmatizer:
        lemmatizer = WordNetLemmatizer()
    else:
        stemmer = SnowballStemmer("english")
    
    for row in bp_data.itertuples(index=True, name='Doc'):
        text = row.text
        
        # remove numbers
        text = re.sub(r'\d+', '', text)
        
        text_words = tokenizer.tokenize(text)
        
        if use_lemmatizer:
            text_words = [lemmatizer.lemmatize(word, pos="v").lower() for word in text_words
                          if word not in string.punctuation and word.lower() not in en_stop]
        else:
            text_words = [stemmer.stem(word).lower() for word in text_words
                         if word not in string.punctuation and word.lower() not in en_stop]
        
        preproccessed_docs.append({'id': row.Index, 'words': text_words})
    
    return preproccessed_docs

In [69]:
preproccessed_docs = preprocess_docs()

In [70]:
def get_term_by_document_frequency(preprocessed_docs):
    document_by_term = {}
    
    for doc_data in preprocessed_docs:
        doc_id = doc_data['id']
        doc_words = doc_data['words']
        
        document_by_term[doc_id] = {
            'total_words': len(doc_words)
        }
        
        
        for word in set(doc_data['words']):
            document_by_term[doc_id][word] = doc_words.count(word)

    df = pd.DataFrame(document_by_term)
    
    return df

In [71]:
df_frequency = get_term_by_document_frequency(preproccessed_docs)

In [73]:
df_frequency

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
total_words,1121.0,699.0,1241.0,713.0,1267.0,1612.0,585.0,503.0,524.0,1626.0,...,3394.0,673.0,1042.0,433.0,795.0,887.0,1011.0,424.0,625.0,1032.0
response,3.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fall,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
benefit,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,2.0,6.0,NaN,NaN,NaN,NaN,NaN
comparison,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
blocky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
initialisation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
sanderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [292]:
def get_tf_idf(df_frequency):
    df = df_frequency.copy()
    # tf := word frequency / total frequency
    df = df.drop('total_words', inplace=False)[:] / df.loc['total_words']
    
    # idf := log ( len(all_documents) / len(documents_containing_word) )
    
    corpus_size = df.shape[1]

    # number of non-zero cols + 1 to avoid division by zero
    df['doc_frequency'] = df.fillna(0).astype(bool).sum(axis=1) + 1 
    
    df['idf'] = np.log( corpus_size / df['doc_frequency'] )
    # tf-idf := tf * idf
    _cols = df.columns.difference(['idf', 'doc_frequency'])
    df[_cols] = df[_cols].multiply(df["idf"], axis="index")
    
    df.drop(columns=['doc_frequency', 'idf'], inplace=True)
    
    return df

In [293]:
df_tf_idf = get_tf_idf(df_frequency)
display(df_tf_idf)

0      0.001576
1           NaN
2           NaN
3           NaN
4      0.003215
         ...   
332         NaN
333    0.002335
334         NaN
335         NaN
336    0.000542
Name: ai, Length: 337, dtype: float64

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
plus,0.001800,NaN,NaN,NaN,0.001573,NaN,NaN,NaN,NaN,0.001245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
successful,0.001556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002784,NaN
opinions,0.002497,NaN,NaN,NaN,0.002183,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.013015,NaN,NaN,0.002774,NaN,NaN,NaN
subscribe,0.002641,0.004237,NaN,NaN,NaN,NaN,NaN,0.005891,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
want,0.000668,0.001071,0.00182,0.003152,0.001751,0.000447,0.000644,0.001490,NaN,0.001386,...,0.000626,0.000551,0.001061,NaN,NaN,0.002493,0.001855,0.000887,0.000598,0.001033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004674
inexplicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004674
illusion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004674
wavy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004674
